In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm_notebook
import IPython
from sklearn.preprocessing import StandardScaler
import random

Reading in Data

In [ ]:
"""Emily's reading in files"""
data_dir = os.getcwd() + '/data/nfl-big-data-bowl-2024/'
players = pd.read_csv(data_dir + "players.csv")
week_1 = pd.read_csv(data_dir + "tracking_week_1.csv")
week_2 = pd.read_csv(data_dir + "tracking_week_2.csv")
week_3 = pd.read_csv(data_dir + "tracking_week_3.csv")
week_4 = pd.read_csv(data_dir + "tracking_week_4.csv")
week_5 = pd.read_csv(data_dir + "tracking_week_5.csv")
week_6 = pd.read_csv(data_dir + "tracking_week_6.csv")
week_7 = pd.read_csv(data_dir + "tracking_week_7.csv")
week_8 = pd.read_csv(data_dir + "tracking_week_8.csv")
week_9 = pd.read_csv(data_dir + "tracking_week_9.csv")
plays = pd.read_csv(data_dir + "plays.csv")
tackles = pd.read_csv(data_dir + "tackles.csv")

weeks_list = [week_1, week_2, week_3, week_4, week_5, week_6, week_7, week_8, week_9]
week = pd.concat(weeks_list)

# join player positioning information onto a week's worth of tracking data 
week = week.merge(players.loc[:, ['nflId', 'position']], how='left')

In [2]:
#Katie

# Read In csvs
plays =  pd.read_csv("plays.csv")
players = pd.read_csv("players.csv")
week_1 = pd.read_csv("tracking_week_1.csv")
week_2 = pd.read_csv("tracking_week_2.csv")
week_3 = pd.read_csv("tracking_week_3.csv")
week_4 = pd.read_csv("tracking_week_4.csv")
week_5 = pd.read_csv("tracking_week_5.csv")
week_6 = pd.read_csv("tracking_week_6.csv")
week_7 = pd.read_csv("tracking_week_7.csv")
week_8 = pd.read_csv("tracking_week_8.csv")
week_9 = pd.read_csv("tracking_week_9.csv")
tackles = pd.read_csv("tackles.csv")

weeks_list = [week_1, week_2, week_3, week_4, week_5, week_6, week_7, week_8, week_9]
week = pd.concat(weeks_list)

week = week.merge(players.loc[:, ['nflId', 'position']], how='left')
week.shape

(12187398, 18)

In [ ]:
#Matts

def load_dataset(dataset_name):
    """
    Download a specific dataset from data directory.

    Parameters:
    - dataset_name: Name of the dataset to load
    """
    return pd.read_csv(f"C:\\Users\\mattd\\Documents\\GitHub\\big-data-bowl-2024\\data\\{dataset_name}.csv")

# Read In csvs
plays = load_dataset("plays")
players = load_dataset("players")
week_1 = load_dataset("tracking_week_1")
week_2 = load_dataset("tracking_week_2")
week_3 = load_dataset("tracking_week_3")
week_4 = load_dataset("tracking_week_4")
week_5 = load_dataset("tracking_week_5")
week_6 = load_dataset("tracking_week_6")
week_7 = load_dataset("tracking_week_7")
week_8 = load_dataset("tracking_week_8")
week_9 = load_dataset("tracking_week_9")
tackles = load_dataset("tackles")

weeks_list = [week_1, week_2, week_3, week_4, week_5, week_6, week_7, week_8, week_9]
week = pd.concat(weeks_list)

week = week.merge(players.loc[:, ['nflId', 'position']], how='left')
week.shape

Creating the necessary csv to merge back in with output of our model

In [3]:
weeks_upload= pd.concat(weeks_list)
week_with_ids = weeks_upload.merge(players.loc[:, ['nflId', 'position']], how='inner')
week_sorted = week_with_ids.sort_values(['gameId','playId', 'frameId', 'club', 'position', 'jerseyNumber'])
final_upload_week = week_sorted[['gameId','playId', 'frameId', 'nflId', 'position', 'jerseyNumber']]
#final_upload_week.to_csv('all_ids_per_frame.csv', index=False)


Creating df with input and output data

In [8]:
# get distance from ball
ball = week.groupby(['gameId','playId', 'frameId']).apply(lambda g: g[g['club'] == 'football'])[['x','y']]
week_and_ball = week.merge(ball, on = ['gameId','playId', 'frameId'], how = 'left', suffixes=('','_ball') )
week_and_ball['distance_to_ball'] = np.sqrt((week_and_ball["x"] - week_and_ball["x_ball"]) ** 2 + (week_and_ball["y"] - week_and_ball["y_ball"]) ** 2)
print('1')


#sort by position and jerseyNumber
week_and_ball = week_and_ball.sort_values(['gameId','playId', 'frameId', 'club', 'position', 'jerseyNumber'])

#merge with tackles
input = week_and_ball.copy()
input = input.merge(tackles[['gameId', 'playId', 'nflId', 'tackle', 'assist']], on = ['gameId', 'playId', 'nflId'], how = 'left')
input['tackle'] = input['tackle'].fillna(0)
input['assist'] = input['assist'].fillna(0)

print('2')


#get order of players (0-10)
input['number'] = input.groupby(['gameId','playId', 'frameId', 'club']).cumcount()
input = input[input['club'] != 'football']
# check if needed
input = input.sort_values(by = ['gameId', 'playId', 'frameId', 'position', 'jerseyNumber'])
input = input.reset_index(drop = True)

# merge with plays
input_transform = input.copy()
#check
input_transform = input_transform.reset_index(drop = True)
plays = plays[['gameId', 'playId', 'possessionTeam', 'defensiveTeam']]
all_players = input.merge(plays, on = ['gameId', 'playId'])



print('3')

# offense
offense = all_players[all_players['possessionTeam'] == all_players['club']]
offense = offense.rename(columns={c: 'o_'+c for c in offense.columns if c in ['x', 'y', 's', 'a', 'dis', 'o', 'dir', 'distance_to_ball']})

o_table = pd.pivot_table(offense, values=['o_x', 'o_y', 'o_s', 'o_a', 'o_dis', 'o_o', 'o_dir', 'o_distance_to_ball'], index=['gameId', 'playId', 'frameId'],
                                                                     #,'position', 'jerseyNumber'],
                       columns=['number'], aggfunc="mean")
o_table.columns = o_table.columns.map('{0[0]}_{0[1]}'.format) 
o_table = o_table.reset_index(['gameId','playId','frameId'])

# defense - includes output data
defense = all_players[all_players['defensiveTeam'] == all_players['club']]
#defense['nfl_ids'] = defense.groupby(['gameId', 'playId'])['nflId'].unique()

defense = defense.rename(columns={c: 'd_'+c for c in defense.columns if c in ['x', 'y', 's', 'a', 'dis', 'o', 'dir', 'distance_to_ball']})

d_table = pd.pivot_table(defense, values=['d_x', 'd_y', 'd_s', 'd_a', 'd_dis', 'd_o', 'd_dir', 'd_distance_to_ball', 'tackle', 'assist'], index=['gameId', 'playId', 'frameId'],
                                                                     #,'position', 'jerseyNumber'],
                       columns=['number'], aggfunc="mean")
d_table.columns = d_table.columns.map('{0[0]}_{0[1]}'.format) 
d_table = d_table.reset_index(['gameId','playId','frameId'])

# combinate offense and defense
input_output_df = o_table.merge(d_table, on = ['gameId', 'playId', 'frameId'], how = 'right')

# calculate tackle credit 
tackle_cols = ['tackle_0', 'tackle_1', 'tackle_2', 'tackle_3', 'tackle_4', 'tackle_5', 'tackle_6', 'tackle_7', 'tackle_8', 'tackle_9', 'tackle_10']
assist_cols = ['assist_0', 'assist_1', 'assist_2', 'assist_3', 'assist_4', 'assist_5', 'assist_6', 'assist_7', 'assist_8', 'assist_9', 'assist_10']
tackle_assist_cols = tackle_cols + assist_cols

input_output_df['total_tacklers'] = input_output_df[tackle_assist_cols].sum(axis=1)
input_output_df['tackle_11'] = np.where(input_output_df['total_tacklers'] == 0 , 1, 0)

input_output_df['tackle_0'] = input_output_df['tackle_0'] + input_output_df['assist_0']
input_output_df['tackle_1'] = input_output_df['tackle_1'] + input_output_df['assist_1']
input_output_df['tackle_2'] = input_output_df['tackle_2'] + input_output_df['assist_2']
input_output_df['tackle_3'] = input_output_df['tackle_3'] + input_output_df['assist_3']
input_output_df['tackle_4'] = input_output_df['tackle_4'] + input_output_df['assist_4']
input_output_df['tackle_5'] = input_output_df['tackle_5'] + input_output_df['assist_5']
input_output_df['tackle_6'] = input_output_df['tackle_6'] + input_output_df['assist_6']
input_output_df['tackle_7'] = input_output_df['tackle_7'] + input_output_df['assist_7']
input_output_df['tackle_8'] = input_output_df['tackle_8'] + input_output_df['assist_8']
input_output_df['tackle_9'] = input_output_df['tackle_9'] + input_output_df['assist_9']
input_output_df['tackle_10'] = input_output_df['tackle_10'] + input_output_df['assist_10']

input_output_df.loc[:, tackle_cols] = input_output_df.loc[:, tackle_cols].div(input_output_df['total_tacklers'], axis=0).fillna(0)
# drop unnecessary columns
input_output_df.drop(assist_cols, axis=1, inplace=True)
input_output_df.drop('total_tacklers', axis=1, inplace=True)

1
2
3


In [ ]:
Distance to ball adding -- Katie 

In [9]:
# List of columns representing the distance of each tackler from the ball
distance_columns = [
    'd_distance_to_ball_0', 'd_distance_to_ball_1', 'd_distance_to_ball_2',
    'd_distance_to_ball_3', 'd_distance_to_ball_4', 'd_distance_to_ball_5',
    'd_distance_to_ball_6', 'd_distance_to_ball_7', 'd_distance_to_ball_8',
    'd_distance_to_ball_9', 'd_distance_to_ball_10'
]

# Identify the tackler closest to the ball for each frame
#This function is used to find the column name with the minimum value for each row, which indicates the closest tackler for that frame
input_output_df['closest_tackler'] = input_output_df[distance_columns].idxmin(axis=1)

# Extract the tackler number from the column name and create a new column
input_output_df['closest_tackler'] = input_output_df['closest_tackler'].str.extract('(\d+)').astype(int)


In [10]:
#reorder so it is not at the end 

# Identify the index of 'tackle_0' column
tackle_0_index = input_output_df.columns.get_loc("o_a_0")

# Create a new order for columns
# Get all columns up to 'tackle_0', add 'closest_tackler', and then the rest
new_column_order = input_output_df.columns[:tackle_0_index].tolist() + ['closest_tackler'] + input_output_df.columns[tackle_0_index:-1].tolist()

# Reorder the DataFrame
input_output_df = input_output_df[new_column_order]


In [19]:
# See if closer player is same or differnt then tackler 
def check_tackler(row):
    closest_tackler = int(row['closest_tackler'])  # Get the closest tackler number as an integer

    half_tacklers = []

    # Iterate through each tackler column and check if any have a value of 1 or 0.5
    for i in range(12):  
        tackler_column = f'tackle_{i}'

        # Check if the tackler has a value of 1 and is the closest
        if row[tackler_column] == 1 and closest_tackler == i:
            return True
        
        # If the tackler has a value of 0.5, add it to the list of half tacklers
        if row[tackler_column] == 0.5:
            half_tacklers.append(i)
    
    # If the closest tackler is one of those with a value of 0.5, return True
    if closest_tackler in half_tacklers:
        return True

    return False  # If no matching tackler is found, return False

# Apply the function to the DataFrame
input_output_df['is_closest_tackler_true'] = input_output_df.apply(check_tackler, axis=1)


In [22]:
# checker for all 

input_output_df['is_closest_tackler_true'] = input_output_df.apply(check_tackler, axis=1)

print(input_output_df[['closest_tackler', 'tackle_0', 'tackle_1', 'tackle_2', 'tackle_3', 'tackle_4', 'tackle_5', 'tackle_6', 'tackle_7', 'tackle_8', 'tackle_9', 'tackle_10', 'tackle_11', 'is_closest_tackler_true']].head())


   closest_tackler  tackle_0  tackle_1  tackle_2  tackle_3  tackle_4  \
0                8       0.0       1.0       0.0       0.0       0.0   
1                7       0.0       1.0       0.0       0.0       0.0   
2                1       0.0       1.0       0.0       0.0       0.0   
3                1       0.0       1.0       0.0       0.0       0.0   
4                1       0.0       1.0       0.0       0.0       0.0   

   tackle_5  tackle_6  tackle_7  tackle_8  tackle_9  tackle_10  tackle_11  \
0       0.0       0.0       0.0       0.0       0.0        0.0          0   
1       0.0       0.0       0.0       0.0       0.0        0.0          0   
2       0.0       0.0       0.0       0.0       0.0        0.0          0   
3       0.0       0.0       0.0       0.0       0.0        0.0          0   
4       0.0       0.0       0.0       0.0       0.0        0.0          0   

   is_closest_tackler_true  
0                    False  
1                    False  
2                

In [13]:
# checker for .5 tackler and true

input_output_df['is_closest_tackler_true'] = input_output_df.apply(check_tackler, axis=1)

# Filter for rows where any tackler column has a value of 0.5
half_tackler_mask = (input_output_df[[f'tackle_{i}' for i in range(11)]] == 0.5).any(axis=1)
half_tackler_df = input_output_df[half_tackler_mask]

#  'is_closest_tackler_true' is True
true_half_tackler_df = half_tackler_df[half_tackler_df['is_closest_tackler_true']]

print(true_half_tackler_df[['closest_tackler', 'tackle_0', 'tackle_1', 'tackle_2', 'tackle_3', 'tackle_4', 'tackle_5', 'tackle_6', 'tackle_7', 'tackle_8', 'tackle_9', 'tackle_10', 'is_closest_tackler_true']])


        closest_tackler  tackle_0  tackle_1  tackle_2  tackle_3  tackle_4  \
147                   4       0.0       0.0       0.0       0.0       0.5   
148                   4       0.0       0.0       0.0       0.0       0.5   
149                   4       0.0       0.0       0.0       0.0       0.5   
150                   4       0.0       0.0       0.0       0.0       0.5   
151                   4       0.0       0.0       0.0       0.0       0.5   
...                 ...       ...       ...       ...       ...       ...   
529595                6       0.0       0.0       0.0       0.0       0.0   
529596                6       0.0       0.0       0.0       0.0       0.0   
529597                6       0.0       0.0       0.0       0.0       0.0   
529598                6       0.0       0.0       0.0       0.0       0.0   
529599                6       0.0       0.0       0.0       0.0       0.0   

        tackle_5  tackle_6  tackle_7  tackle_8  tackle_9  tackle_10  \
147 

In [15]:
# half tackles false 

input_output_df['is_closest_tackler_true'] = input_output_df.apply(check_tackler, axis=1)

# Filter for rows where any tackler column has a value of 0.5
half_tackler_mask = (input_output_df[[f'tackle_{i}' for i in range(11)]] == 0.5).any(axis=1)
half_tackler_df = input_output_df[half_tackler_mask]

# Filter for instances where 'is_closest_tackler_true' is False
false_half_tackler_df = half_tackler_df[~half_tackler_df['is_closest_tackler_true']]

print(false_half_tackler_df[['closest_tackler', 'tackle_0', 'tackle_1', 'tackle_2', 'tackle_3', 'tackle_4', 'tackle_5', 'tackle_6', 'tackle_7', 'tackle_8', 'tackle_9', 'tackle_10', '' 'is_closest_tackler_true']])


        closest_tackler  tackle_0  tackle_1  tackle_2  tackle_3  tackle_4  \
134                   8       0.0       0.0       0.0       0.0       0.5   
135                   8       0.0       0.0       0.0       0.0       0.5   
136                   8       0.0       0.0       0.0       0.0       0.5   
137                   8       0.0       0.0       0.0       0.0       0.5   
138                   8       0.0       0.0       0.0       0.0       0.5   
...                 ...       ...       ...       ...       ...       ...   
529587                3       0.0       0.0       0.0       0.0       0.0   
529588                3       0.0       0.0       0.0       0.0       0.0   
529589                3       0.0       0.0       0.0       0.0       0.0   
529590                3       0.0       0.0       0.0       0.0       0.0   
529591                3       0.0       0.0       0.0       0.0       0.0   

        tackle_5  tackle_6  tackle_7  tackle_8  tackle_9  tackle_10  \
134 

In [14]:
true_count = input_output_df['is_closest_tackler_true'].sum()  # True
false_count = len(input_output_df) - true_count  # False count is the total length minus the number of True values

print("Number of True values:", true_count)
print("Number of False values:", false_count)

Number of True values: 224423
Number of False values: 305456


In [24]:
# Calculate the percentage of False in 'is_closest_tackler_true' for each play
play_summary = input_output_df.groupby(['gameId', 'playId']).agg(
    total_frames=pd.NamedAgg(column='frameId', aggfunc='count'),
    false_frames=pd.NamedAgg(column='is_closest_tackler_true', aggfunc=lambda x: (x==False).sum()),
    last_frame_tackle=pd.NamedAgg(column='is_closest_tackler_true', aggfunc='last')
).reset_index()

# Calculate the percentage of frames that are False
play_summary['false_percentage'] = (play_summary['false_frames'] / play_summary['total_frames']) * 100

# Step 2: Filter plays where at least 70% of the frames are False, and the last frame is True
target_plays = play_summary[(play_summary['false_percentage'] >= 70) & (play_summary['last_frame_tackle'] == True)]

# Print the 'gameId' and 'playId' of these plays
for index, row in target_plays.iterrows():
    print(f"Game ID: {row['gameId']}, Play ID: {row['playId']} - Opportunity found where the tackler wasn't always closest but ended up tackling.")

# You might want to return or further process target_plays depending on your needs


Game ID: 2022090800, Play ID: 343 - Opportunity found where the tackler wasn't always closest but ended up tackling.
Game ID: 2022090800, Play ID: 896 - Opportunity found where the tackler wasn't always closest but ended up tackling.
Game ID: 2022090800, Play ID: 1254 - Opportunity found where the tackler wasn't always closest but ended up tackling.
Game ID: 2022090800, Play ID: 2360 - Opportunity found where the tackler wasn't always closest but ended up tackling.
Game ID: 2022090800, Play ID: 2485 - Opportunity found where the tackler wasn't always closest but ended up tackling.
Game ID: 2022090800, Play ID: 3262 - Opportunity found where the tackler wasn't always closest but ended up tackling.
Game ID: 2022090800, Play ID: 3304 - Opportunity found where the tackler wasn't always closest but ended up tackling.
Game ID: 2022090800, Play ID: 3362 - Opportunity found where the tackler wasn't always closest but ended up tackling.
Game ID: 2022091100, Play ID: 57 - Opportunity found where

In [27]:
# CHECKER SO U CAN SEE THE PLAY OUTPUTTED 

#Here, i just used the fiest play in the above list, just chnage how u want it
# Filter for the specific Game ID and Play ID
specific_play = input_output_df[(input_output_df['gameId'] == 2022090800) & (input_output_df['playId'] == 343)]

with pd.option_context('display.max_columns', None):  
    print(specific_play)  # print the WHOLE play

         gameId  playId  frameId  closest_tackler  o_a_0  o_a_1  o_a_2  o_a_3  \
355  2022090800     343        1                5   0.00   0.00   0.37   0.04   
356  2022090800     343        2                5   0.28   0.00   0.33   0.08   
357  2022090800     343        3                5   0.27   0.00   0.32   0.12   
358  2022090800     343        4                5   0.10   0.00   0.24   0.14   
359  2022090800     343        5                5   0.24   0.35   0.14   0.13   
360  2022090800     343        6                5   0.65   0.90   0.24   0.11   
361  2022090800     343        7                5   1.03   1.50   0.36   0.07   
362  2022090800     343        8                5   1.33   2.00   0.42   0.06   
363  2022090800     343        9                5   1.40   2.35   0.39   0.04   
364  2022090800     343       10                5   1.22   2.34   0.52   0.02   
365  2022090800     343       11                5   1.20   2.02   0.40   0.02   
366  2022090800     343     

In [ ]:
just_input = input_output_df.iloc[:, 3:-12]
just_input

In [ ]:
just_output = input_output_df.iloc[:, -12:]
just_output

In [ ]:
scaler = StandardScaler()
normalized_input = pd.DataFrame(scaler.fit_transform(just_input), columns=just_input.columns)
normalized_input

In [ ]:
#concat 
normalized_input_output_df = pd.concat([input_output_df[['gameId', 'playId', 'frameId']], normalized_input, just_output], axis = 1)

In [ ]:
# def last_5_converter(df):
#   final_input = pd.DataFrame(columns=['four', 'three', 'two', 'one', 'current', 'tackle_1','tackle_2','tackle_3','tackle_4','tackle_5','tackle_6','tackle_7','tackle_8','tackle_9','tackle_10','tackle_11', 'tackle_12'])
#   for idx, row in df.iterrows():
#     if row['frameId'] < 5:
#       current = df.iloc[idx, 4:-12].tolist()
#       previous_frames = [np.zeros(17) for _ in range(4)]
  #     #print("FrameID",row['frameId'])
  #     # Use a loop to assign the lists
  #     for i in range(1, int(row['frameId'])):
  #         #print(3-i)
  #         previous_frames[4-i] = df.iloc[idx - i, 4:-12].tolist()

  #     new_row = [previous_frames[0], previous_frames[1], previous_frames[2], previous_frames[3], current, df.at[idx,'tackle_0'], df.at[idx,'tackle_1'], df.at[idx,'tackle_2'], df.at[idx,'tackle_3'], df.at[idx,'tackle_4'], df.at[idx,'tackle_5'], df.at[idx,'tackle_6'], df.at[idx,'tackle_7'], df.at[idx,'tackle_8'], df.at[idx,'tackle_9'], df.at[idx,'tackle_10'], df.at[idx, 'tackle_11']]
  #     final_input.loc[len(final_input.index)] = new_row 
  #   else:
  #     current = df.iloc[idx, 4:-12].tolist()
  #     one = df.iloc[idx - 1, 4:-12].tolist()
  #     two = df.iloc[idx - 2, 4:-12].tolist()
  #     three = df.iloc[idx - 3, 4:-12].tolist()
  #     four = df.iloc[idx - 4, 4:-12].tolist()
  #     new_row = [four, three, two, one, current, df.at[idx,'tackle_0'], df.at[idx,'tackle_1'], df.at[idx,'tackle_2'], df.at[idx,'tackle_3'], df.at[idx,'tackle_4'], df.at[idx,'tackle_5'], df.at[idx,'tackle_6'], df.at[idx,'tackle_7'], df.at[idx,'tackle_8'], df.at[idx,'tackle_9'], df.at[idx,'tackle_10'], df.at[idx, 'tackle_11']]
  #     # Assuming final_input.columns contains the correct column names
  #     new_row_series = pd.Series(new_row, index=final_input.columns)
  #     # Concatenate along axis=1
  #     final_input.loc[len(final_input.index)] = new_row 
  #     #final_input = pd.concat([final_input,pd.Series(new_row, index=final_input.columns)], axis = 1, ignore_index=True)
  # return final_input

In [ ]:
def last_5_converter(df):
    rows = []
    for idx, row in tqdm_notebook(df.iterrows()):
        current = df.iloc[idx, 3:-12].tolist()
        tackles = [df.at[idx, f'tackle_{i}'] for i in range(12)]
        #ids = [df.at[idx, f'{i}'] for i in ['gameId', 'playId', 'frameId']]
        if row['frameId'] < 5:
            previous_frames = [np.zeros(175) for _ in range(4)]
            for i in range(1, int(row['frameId'])):
                previous_frames[4-i] = df.iloc[idx - i, 3:-12].tolist()
        else:
            previous_frames = [df.iloc[idx - i, 3:-12].tolist() for i in range(1, 5)]
        new_row = previous_frames + [current] + tackles
        rows.append(new_row)
    final_input = pd.DataFrame(rows, columns=['four', 'three', 'two', 'one', 'current', 'tackle_1','tackle_2','tackle_3','tackle_4','tackle_5','tackle_6','tackle_7','tackle_8','tackle_9','tackle_10','tackle_11', 'tackle_12'])
    return final_input

In [ ]:
model_input_no_id = last_5_converter(normalized_input_output_df)

In [ ]:
model_input_no_id

In [ ]:
model_input = pd.concat([input_output_df[['gameId', 'playId', 'frameId']], model_input_no_id], axis = 1)

In [ ]:
model_input

In [ ]:
# average number of frames per play - can work backwards (100k or 200k / 44 to get approx num of plays to grab)
model_input.groupby(['gameId', 'playId'])[['frameId']].agg('count')['frameId'].median()

In [ ]:
random_sample_100k = random.choices(model_input[['gameId', 'playId']].values.tolist(), k=2250)
random_sample_200k = random.choices(model_input[['gameId', 'playId']].values.tolist(), k=5500)

In [ ]:
random_sample_100k = [tuple(i) for i in random_sample_100k]
model_input_100k_random_sample = model_input[model_input[['gameId', 'playId']].apply(tuple, axis = 1).isin(random_sample_100k)]

In [ ]:
model_input_100k_random_sample

In [ ]:
random_sample_200k = [tuple(i) for i in random_sample_200k]
model_input_200k_random_sample = model_input[model_input[['gameId', 'playId']].apply(tuple, axis = 1).isin(random_sample_200k)]

In [ ]:
model_input_200k_random_sample

In [ ]:
model_input_100k_random_sample.to_pickle("model_input_100k_random_sample.pkl")

In [ ]:
model_input_200k_random_sample.to_pickle("model_input_200k_random_sample.pkl")